In [7]:
import numpy as np
import pandas as pd
import requests
import config
import os
import datadotworld as dw
import json
from pandas.io.json import json_normalize
import time

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [2]:
## MAINE DONATIONS TO PRESIDENTIAL CAMPAIGNS ##
#Initialize dataframe collector for itemized contribs
cycle='2020'
dfs=[]
id=int(0)

#Initialize query dict
querydict = {'per_page':'100'
              ,'sort':'contribution_receipt_date'
              ,'api_key':config.api_key
              ,'is_individual':'true'
              ,'two_year_transaction_period':cycle
              ,'recipient_committee_type':'P'
              ,'contributor_state':'ME'
              ,'last_index':[]
              ,'last_contribution_receipt_date':[]
             }

r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()
    
#Last page variables
while r['pagination']['last_indexes'] is not None:
    
    #Store results
    df = json_normalize(r['results'])
    dfs.append(df)
    
    #Declare and assign last date variables
    last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
    last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']   
    #Update dictionary with index dates
    querydict.update([('last_index',last_index)
                    ,('last_contribution_receipt_date',last_date)])
    
    #Get next payload
    r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()

# Concatenate all dfs
itemdf=pd.concat(dfs,sort=False,ignore_index=True)
itemdf=itemdf.drop_duplicates(subset='transaction_id')

In [4]:
#Set ZIPs to 5 digits
itemdf['contributor_zip'] = itemdf['contributor_zip'].str[:5]

#Filter out 00000, Nulls
itemdf = itemdf[(itemdf['contributor_zip'].isnull()==False)
                 & (itemdf['contributor_zip']!='00000')]

#Filter ZIPs outside Maine (03 or 04 start)
itemdf = itemdf[(itemdf['contributor_zip'].str[:2]=='03')
               | (itemdf['contributor_zip'].str[:2]=='04')] 
                

In [10]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
itemdf.to_csv(cwd+'/data/maine-presidential-donations.csv')

In [9]:
#Write to data.world
with dw.open_remote_file('darrenfishell/2020-election-repo','presidential-contributions-from-maine.csv') as w:
    itemdf.to_csv(w,index=False)

In [ ]:
##COLLECT AND WRITE COMMITTEE LOOKUPS##